In [7]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def resize_image(image, target_size):
    target_height, target_width = target_size
    original_height, original_width = image.shape[:2]

    # Calculate scaling factors
    scale_x = target_width / original_width
    scale_y = target_height / original_height

    # Initialize the resized image
    resized_image = np.zeros((target_height, target_width), dtype=np.uint8)

    # Nearest neighbor interpolation
    for y in range(target_height):
        for x in range(target_width):
            px = int(x / scale_x)
            py = int(y / scale_y)
            resized_image[y, x] = image[py, px]

    return resized_image

def zero_pad(image, target_size):
    target_height, target_width = target_size
    original_height, original_width = image.shape[:2]

    # Calculate padding amounts
    pad_left = (target_width - original_width) // 2
    pad_right = target_width - original_width - pad_left
    pad_top = (target_height - original_height) // 2
    pad_bottom = target_height - original_height - pad_top

    # Pad the image
    padded_image = cv2.copyMakeBorder(image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)

    return padded_image

def convolve(image, kernel):
    # Flip the kernel
    kernel = np.flipud(np.fliplr(kernel))

    # Get dimensions of image and kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Get dimensions for padding
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2

    # Pad the image
    padded_image = np.zeros((image_height + 2 * pad_height, image_width + 2 * pad_width))
    padded_image[pad_height:-pad_height, pad_width:-pad_width] = image

    # Initialize the output image
    output = np.zeros_like(image)

    # Convolve the image with the kernel
    for y in range(image_height):
        for x in range(image_width):
            output[y, x] = np.sum(kernel * padded_image[y:y+kernel_height, x:x+kernel_width])

    return output

def gaussian_kernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2) / (2*sigma**2)), (size, size))
    return kernel / np.sum(kernel)

def gaussian_blur(image, size, sigma):
    kernel = gaussian_kernel(size, sigma)
    return convolve(image, kernel)

def fft_magnitude(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return magnitude_spectrum

def main():
    image_paths = ['F:\Sem6\Computer Vision\Lab\Lab6\PNG_images\512\einstein.png', 'F:\Sem6\Computer Vision\Lab\Lab6\PNG_images\512\panda.png']  # Paths to your images
    sizes = [(256, 256), (128, 128), (64, 64), (32, 32), (16, 16)]

    for img_path in image_paths:
        original_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        print("Original image shape:", original_image.shape)

        for target_size in sizes:
            # Resize the image
            resized_image = resize_image(original_image, target_size)

            # Zero pad the resized image
            padded_image = zero_pad(resized_image, (512, 512))

            # Apply Gaussian blur
            blurred_image = gaussian_blur(padded_image, size=5, sigma=1.4)

            # Compute FFT magnitude
            fft_mag = fft_magnitude(blurred_image)

            # Display FFT magnitude
            cv2.imshow(f'FFT Magnitude of {target_size}', fft_mag.astype(np.uint8))

            # Compute PSNR and SSIM
            original_resized = cv2.resize(original_image, target_size)
            psnr = peak_signal_noise_ratio(original_resized, blurred_image)
            ssim = structural_similarity(original_resized, blurred_image)
            print(f"Size: {target_size}, PSNR: {psnr}, SSIM: {ssim}")

            cv2.waitKey(0)
            cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


AttributeError: 'NoneType' object has no attribute 'shape'